In [3]:
import geopandas as gpd 
import pandas as pd
import os
from glob import glob


In [5]:
    state = '08'
    CO_path = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties'
    #CO_pInventory = pInventory.loc[pInventory['STATE']=='08']
    CO_pInventory = pd.read_csv(r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\ColoradoInventory.csv', dtype={'STATE':str,'COUNTY':str})
    CO_pInventory_True = CO_pInventory.loc[CO_pInventory['DATA_PRESENT']==True]
    CO_pInventory_False = CO_pInventory.loc[CO_pInventory['DATA_PRESENT']==False]
    CO_pInventory_False.to_csv(os.path.join(CO_path,'missingParcelData.csv')) # need to get this to work
    fipsList = CO_pInventory_True['COUNTY'].tolist()   
    parcelsPaths = [os.path.join(CO_path,fips) for fips in fipsList]
    parcelsPaths = glob(r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\\*\\')

In [6]:
parcelsPaths

['C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08001\\',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08003\\',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08005\\',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08007\\',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08013\\',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08014\\',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08015\\',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08017\\',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08019\\',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08021\\',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08023\\',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08027\\',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08029\\',
 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08031\\',
 'C:\\

In [37]:
for path in parcelsPaths:
    fips = path.split('\\')[-1]
    if os.path.exists(os.path.join(path,'MHP_'+fips+'.csv')):
        os.remove(os.path.join(path,'MHP_'+fips+'.csv'))
    if os.path.exists(os.path.join(path,'MHP_.csv')):
        os.remove(os.path.join(path,'MHP_.csv'))

In [38]:
OGmhps = pd.read_csv(r'C:\Users\phwh9568\Data\ParcelAtlas\Mobile_Home_Parks\MobileHomeParks.csv', dtype={'COUNTYFIPS':str})
COmhps = OGmhps.loc[OGmhps['COUNTYFIPS'].str.startswith('08')]

In [39]:
for path in parcelsPaths:
    fips = path.split('\\')[-2]
    countyDF = COmhps.loc[COmhps['COUNTYFIPS']==fips].copy()
    columns = ['MHPID','NAME','ADDRESS', 'CITY', 'STATE', 'ZIP', 'STATUS', 'COUNTYFIPS','LATITUDE', 'LONGITUDE']
    renames = ['MHPID','MH_NAME','MH_ADDRESS', 'MH_CITY', 'MH_STATE', 'MH_ZIP', 'MH_STATUS','MH_COUNTY_FIPS','MH_LATITUDE', 'MH_LONGITUDE']
    drops = [c for c in countyDF.columns if c not in columns] 
    renames = dict(zip(columns,renames))
    countyDF.drop(drops,axis=1, inplace=True)
    countyDF.rename(renames, axis='columns',inplace=True)
    countyDF.to_csv(os.path.join(path,'MHP_'+str(fips)+'.csv'))

In [36]:
parcelsPaths[1].split('\\')[-2]

'08003'

In [1]:
co_22_dir = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022'
parcel_dir = r'C:\Users\phwh9568\Data\ParcelAtlas'

Now repeat for COSTAR mhps

In [9]:
costar_path = os.path.join(parcel_dir,'PARKS_W_COSTAR_TRANSACTIONS_2024-01-04.csv')
costar_mhps = pd.read_csv(costar_path)
fipsCodes = pd.read_csv(os.path.join(r'C:\Users\phwh9568\Data\ParcelAtlas', 'fips-by-state.csv'), dtype={'fips':str}, encoding='iso-8859-1')

In [10]:
CO_fips = fipsCodes.loc[fipsCodes['fips'].str.startswith('08')].copy()
CO_fips['propertycounty'] = CO_fips['name'].str.replace(' County','')
costar_mhps = pd.merge(costar_mhps,CO_fips[['propertycounty','fips',]], on='propertycounty')

In [ ]:
costar_mhps

In [12]:
for path in parcelsPaths:
    fips = path.split('\\')[-2]
    countyDF = costar_mhps.loc[costar_mhps['fips']==fips].copy()
    columns = ['property_id', 'propertyname', 'propertycity','propertystate', 'propertycounty', 'propertyzipcode', 'latitude', 'longitude', 'parcelnumber1min','numberofunits', 'fips']
    renames = ['MH_prop_id', 'MH_prop_name', 'MH_prop_city', 'MH_prop_state', 'MH_prop_county', 'MH_prop_zip', 'MH_lat', 'MH_long', 'MH_parcel_num', 'MH_units', 'MH_COUNTY_FIPS']
    drops = [c for c in countyDF.columns if c not in columns] 
    renames = dict(zip(columns,renames))
    countyDF.drop(drops,axis=1, inplace=True)
    countyDF.rename(renames, axis='columns',inplace=True)
    if len(countyDF) > 0:
        countyDF.to_csv(os.path.join(path,'MHP_'+str(fips)+'_COSTAR.csv'))

finalDF = pd.DataFrame()
for path in parcelsPaths:
    fips = path.split('\\')[-2]
    if os.path.exists(os.path.join(path,fips+'.gpkg')):
        county = gpd.read_file(os.path.join(path, fips+'.gpkg'),layer='MH_parc_blk_union')
        finalDF = pd.concat([finalDF,county])
finalDF.to_file(os.path.join(co_22_dir,'Colorado_Final_original.gpkg'),layer='Colorado_Final_original')

finalDF = pd.DataFrame()
for path in parcelsPaths:
    fips = path.split('\\')[-2]
    if os.path.exists(os.path.join(path,fips+'.gpkg')):
        county = gpd.read_file(os.path.join(path, fips+'.gpkg'),layer='MH_parc_blk_union')
        finalDF = pd.concat([finalDF,county])
finalDF.to_file(os.path.join(co_22_dir,'Colorado_Final_original.gpkg'),layer='Colorado_Final_original')